In [1]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
executable_path = {'executable_path': 'c:/util/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Step 1 - Scraping

Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

* Create a Jupyter Notebook file called `mission_to_mars.ipynb` and use this to complete all of your scraping and analysis tasks. The following outlines what you need to scrape.

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

```python
# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."
```

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

article = soup.find('li', class_='slide')

news_title = article.find('h3').text

news_p = article.find('div', class_='article_teaser_body').text

In [4]:
print(news_title)

Curiosity Says Farewell to Mars' Vera Rubin Ridge


In [5]:
print(news_p)

The Mars rover is moving on and has taken its last selfie on the ridge it's explored for more than a year.



### JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

```python
# Example:
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
```

In [6]:
url = 'https://www.jpl.nasa.gov'
search_string = '/spaceimages/?search=&category=Mars'
browser.visit(url + search_string)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#todo: figure out the actual largeize url instead of replacing text
featured_image_url = url + soup.find('section', class_='main_feature').\
     find('div', class_='carousel_items').\
     find('footer').\
     find('a')["data-fancybox-href"].\
     replace('mediumsize', 'largesize').\
     replace('_ip.jpg', '_hires.jpg')

In [7]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19643_hires.jpg


### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

```python
# Example:
mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'
```


In [8]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather_tag = soup.find('div', class_='js-tweet-text-container').find('p', class_='TweetTextSize')
mars_weather = mars_weather_tag.text.replace(mars_weather_tag.a.text, '')

In [9]:
mars_weather

'Sol 2303 (2019-01-28), high -7C/19F, low -73C/-99F, pressure at 8.15 hPa, daylight 06:47-18:54'

### Mars Facts

* Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.


In [10]:
url = 'http://space-facts.com/mars/'
tables = pd.read_html(url)

In [11]:
df = tables[0]
df.columns = ['description', 'value']

In [12]:
#df = tables[0].T
#df.columns = df.iloc[0].str.replace(':', '')
#df = df.iloc[1:]

In [13]:
df

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers



### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]
```

In [14]:
url = 'https://astrogeology.usgs.gov'
search_string = '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url + search_string)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
items = soup.find('div', class_='full-content').\
      find('div', class_='collapsible results').\
      find_all('div', class_='item')

In [16]:
hemisphere_image_urls = []
for item in items:
    hemisphere_image_urls.append(
        {"title": item.find('div', class_='description').h3.text
         , "img_url": f"{url}{item.a['href']}"
        }
    )

In [17]:
for i, url in enumerate(hemisphere_image_urls):
    browser.visit(url['img_url'])
    soup = BeautifulSoup(browser.html, 'html.parser')
    hemisphere_image_urls[i]['img_url'] = soup.find('div', class_='downloads').\
        ul.\
        find_all('li')[1].\
        a['href']

In [18]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]